## Cy Young Winners on Losing Teams
by [Nate Rattner](http://naterattner.com)  
This code scrapes and analyzes the data behind my November 2018 story for FanSided about MLB pitchers to win the [Cy Young Award on losing teams](https://fansided.com/2018/11/13/jacob-degrom-cy-young-award-victory/). If Jacob deGrom wins the 2018 NL award, he will become just the 17th pitcher in baseball history to win the Cy Young on a losing team since the award was created in 1956.

In [2]:
from selenium import webdriver
from bs4 import BeautifulSoup
import pandas as pd
import matplotlib.pyplot as plt

options = webdriver.ChromeOptions()
options.add_argument("headless")
options.add_argument("window-size=1200x600")

### Scraping Team Records from 1956 to 2018

In [3]:
driver = webdriver.Chrome(executable_path='/Users/Rattner/Documents/Personal/cy-young/cy-young/chromedriver')

url_template = "https://www.baseball-reference.com/leagues/MLB/{year}-standings.shtml"
#driver = webdriver.Chrome(executable_path='/Users/Rattner/Documents/Personal/cy-young/cy-young/chromedriver')

# create an empty DataFrame
data_df = pd.DataFrame()


for year in range(1956, 2019):  # for each year
    url = url_template.format(year=year)  # get the url
    driver.get(url)
    html = driver.page_source  # get the html
    soup = BeautifulSoup(html) # create a BS object
    tableSoup = soup.find("table", {"id": "expanded_standings_overall"}) # pull just the table from BS object

    # get our player data
    data_rows = tableSoup.findAll('tr')[1:] 
    team_data = [[td.getText() for td in data_rows[i].findAll(['td','th'])] for i in range(len(data_rows))]
    
    # Turn yearly data into a DataFrame
    year_df = pd.DataFrame(team_data) #Remove second arg and it works. Need to keep fewer column headers (from 2017 table)
    # create and insert the Season column
    year_df.insert(0, 'Season', year)
    
    # Append to the big dataframe
    data_df = data_df.append(year_df, ignore_index=True, sort=True)

In [4]:
data_df.head()

,0,1,2,3,4,5,6,7,8,9,...,18,19,20,21,22,23,24,25,26,Season
0,1,NYY,AL,154,97,57,.630,5.6,4.1,1.5,...,61-40,36-17,49-39,48-18,NaN,NaN,NaN,NaN,NaN,1956
1,2,BRO,NL,154,93,61,.604,4.7,3.9,0.8,...,86-54,7-7,21-23,72-38,NaN,NaN,NaN,NaN,NaN,1956
2,3,MLN,NL,155,92,62,.597,4.6,3.7,0.9,...,79-55,13-7,25-19,67-43,NaN,NaN,NaN,NaN,NaN,1956
3,4,CLE,AL,155,88,66,.571,4.6,3.7,0.8,...,61-45,27-21,37-51,51-15,NaN,NaN,NaN,NaN,NaN,1956
4,5,CIN,NL,155,91,63,.591,5.0,4.2,0.8,...,65-41,26-22,20-24,71-39,NaN,NaN,NaN,NaN,NaN,1956


In [9]:
seasons_data = data_df.set_index('Season').reset_index()

#seasons_data = data_df[[0, 3, 4]] #Selecting only cy young winners
seasons_data.head()

,Season,0,1,2,3,4,5,6,7,8,...,17,18,19,20,21,22,23,24,25,26
0,1956,1,NYY,AL,154,97,57,.630,5.6,4.1,...,18-19,61-40,36-17,49-39,48-18,NaN,NaN,NaN,NaN,NaN
1,1956,2,BRO,NL,154,93,61,.604,4.7,3.9,...,25-21,86-54,7-7,21-23,72-38,NaN,NaN,NaN,NaN,NaN
2,1956,3,MLN,NL,155,92,62,.597,4.6,3.7,...,22-21,79-55,13-7,25-19,67-43,NaN,NaN,NaN,NaN,NaN
3,1956,4,CLE,AL,155,88,66,.571,4.6,3.7,...,23-24,61-45,27-21,37-51,51-15,NaN,NaN,NaN,NaN,NaN
4,1956,5,CIN,NL,155,91,63,.591,5.0,4.2,...,29-22,65-41,26-22,20-24,71-39,NaN,NaN,NaN,NaN,NaN


In [11]:
seasons_data = seasons_data.iloc[: , [0,1,2,3,4,5,6,7]] #Selecting only W-L data
seasons_data.columns = ['Season', 'Rk', 'Team', 'Lg', 'G', 'W', 'L', 'W-L%'] #Name columns
seasons_data.head()

,Season,Rk,Team,Lg,G,W,L,W-L%
0,1956,1,NYY,AL,154,97,57,.630
1,1956,2,BRO,NL,154,93,61,.604
2,1956,3,MLN,NL,155,92,62,.597
3,1956,4,CLE,AL,155,88,66,.571
4,1956,5,CIN,NL,155,91,63,.591


In [12]:
seasons_data['Season']=seasons_data['Season'].apply(str) #Change data type to string
seasons_data.dtypes

Season    object
Rk        object
Team      object
Lg        object
G         object
W         object
L         object
W-L%      object
dtype: object

### Loading Data of Cy Young Winners

In [13]:
cy_young_winners = pd.read_csv('cy-young-winners.csv') #Load Cy Young winners CSV

#cy_young_winners[cy_young_winners['Season'].isnull()]
cy_young_winners['Season']=cy_young_winners['Season'].apply(str)
#cy_young_winners.head()
cy_young_winners.dtypes

Season     object
League     object
Player     object
Team       object
W           int64
L           int64
ERA       float64
SO          int64
SV        float64
dtype: object

### Merging the Team Record and Cy Young Winenrs Datasets

In [14]:
merged_data = seasons_data.merge(cy_young_winners, left_on=['Season','Team'], right_on=['Season','Team'], how='inner', indicator=True) #Merging on Season and Team
merged_data.head()

,Season,Rk,Team,Lg,G,W_x,L_x,W-L%,League,Player,W_y,L_y,ERA,SO,SV,_merge
0,1956,2,BRO,NL,154,93,61,.604,Both,Don Newcombe,27,7,3.06,139,NaN,both
1,1957,2,MLN,NL,155,95,59,.617,Both,Warren Spahn,21,11,2.69,111,NaN,both
2,1958,1,NYY,AL,155,92,62,.597,Both,Bob Turley,21,7,2.97,168,NaN,both
3,1959,1,CHW,AL,156,94,60,.610,Both,Early Wynn,22,10,3.17,179,NaN,both
4,1960,2,PIT,NL,155,95,59,.617,Both,Vern Law,20,9,3.08,120,NaN,both


In [15]:
merged_data.rename(columns={'W_x':'Team W','L_x':'Team L','W_y':'Player W','L_y':'Player L','W-L%':'Team W-L%' }, inplace=True) #Renaming columns
merged_data.head()

,Season,Rk,Team,Lg,G,Team W,Team L,Team W-L%,League,Player,Player W,Player L,ERA,SO,SV,_merge
0,1956,2,BRO,NL,154,93,61,.604,Both,Don Newcombe,27,7,3.06,139,NaN,both
1,1957,2,MLN,NL,155,95,59,.617,Both,Warren Spahn,21,11,2.69,111,NaN,both
2,1958,1,NYY,AL,155,92,62,.597,Both,Bob Turley,21,7,2.97,168,NaN,both
3,1959,1,CHW,AL,156,94,60,.610,Both,Early Wynn,22,10,3.17,179,NaN,both
4,1960,2,PIT,NL,155,95,59,.617,Both,Vern Law,20,9,3.08,120,NaN,both


In [16]:
merged_data.sort_values(by='Team W-L%').head(10) #Sort by Team W-L%. 
#The 2010 Seattle Mariners are the worst team in MLB history to have a Cy Young Award-winning pitcher

,Season,Rk,Team,Lg,G,Team W,Team L,Team W-L%,League,Player,Player W,Player L,ERA,SO,SV,_merge
99,2010,29,SEA,AL,162,61,101,.377,AL,Felix Hernandez,13,12,2.27,232,NaN,both
23,1972,23,PHI,NL,156,59,97,.378,NL,Steve Carlton,27,10,1.97,310,NaN,both
97,2009,26,KCR,AL,162,65,97,.401,AL,Zack Greinke,16,8,2.16,242,NaN,both
95,2008,24,SFG,NL,162,72,90,.444,NL,Tim Lincecum,18,5,2.62,265,NaN,both
31,1976,19,SDP,NL,162,73,89,.451,NL,Randy Jones,22,14,2.74,93,NaN,both
103,2012,21,NYM,NL,162,74,88,.457,NL,R.A. Dickey,20,6,2.73,230,NaN,both
71,1996,22,TOR,AL,162,74,88,.457,AL,Pat Hentgen,20,10,3.22,177,NaN,both
22,1972,17,CLE,AL,156,72,84,.462,AL,Gaylord Perry,24,16,1.92,234,NaN,both
73,1997,22,TOR,AL,162,76,86,.469,AL,Roger Clemens,21,7,2.05,292,NaN,both
19,1970,15,STL,NL,162,76,86,.469,NL,Bob Gibson,23,7,3.12,274,NaN,both


In [17]:
full_data = merged_data.append({'Season': 2018, 'Rk': 20, 'Team': 'NYM', 'Lg': 'NL', 'G': 162, 'Team W': 77, 'Team L': 85, 'Team W-L%': '.475', 'League': 'NL', 'Player': 'Jacob deGrom', 'Player W': 10, 'Player L': 9, 'ERA': 1.70, 'SO': 269,}, ignore_index=True)
#Add Jacob deGrom as hypothetical 2018 NL Cy Young winner to dataframe

In [18]:
full_data.sort_values(by='Team W-L%').head(15)

,Season,Rk,Team,Lg,G,Team W,Team L,Team W-L%,League,Player,Player W,Player L,ERA,SO,SV,_merge
99,2010,29,SEA,AL,162,61,101,.377,AL,Felix Hernandez,13,12,2.27,232,NaN,both
23,1972,23,PHI,NL,156,59,97,.378,NL,Steve Carlton,27,10,1.97,310,NaN,both
97,2009,26,KCR,AL,162,65,97,.401,AL,Zack Greinke,16,8,2.16,242,NaN,both
95,2008,24,SFG,NL,162,72,90,.444,NL,Tim Lincecum,18,5,2.62,265,NaN,both
31,1976,19,SDP,NL,162,73,89,.451,NL,Randy Jones,22,14,2.74,93,NaN,both
71,1996,22,TOR,AL,162,74,88,.457,AL,Pat Hentgen,20,10,3.22,177,NaN,both
103,2012,21,NYM,NL,162,74,88,.457,NL,R.A. Dickey,20,6,2.73,230,NaN,both
22,1972,17,CLE,AL,156,72,84,.462,AL,Gaylord Perry,24,16,1.92,234,NaN,both
73,1997,22,TOR,AL,162,76,86,.469,AL,Roger Clemens,21,7,2.05,292,NaN,both
19,1970,15,STL,NL,162,76,86,.469,NL,Bob Gibson,23,7,3.12,274,NaN,both


In [19]:
full_data.to_csv('cy_young_winners.csv', encoding='utf-8', index=False) #Save data to CSV